<a href="https://colab.research.google.com/github/bankehsaz/Holt-Winters/blob/main/Holt_Winters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import numpy as np
import pandas as pd
import plotly.express as px
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [2]:
# Read the data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/international-airline-passengers.csv', index_col='Month', parse_dates=True)
df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [3]:
# Check NaN
df.isna().sum()

Passengers    0
dtype: int64

In [4]:
# Plot 
px.line(df, x=df.index, y='Passengers')

In [5]:
# Exponentially Weighted Moving Average (EWMA)
df['EWMA'] = df['Passengers'].ewm(alpha=0.2, adjust=False).mean()
df.head()

,Passengers,EWMA
Month,,
1949-01-01,112,112.0000
1949-02-01,118,113.2000
1949-03-01,132,116.9600
1949-04-01,129,119.3680
1949-05-01,121,119.6944


In [6]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'EWMA'])

# **SimpleExpSmoothing Model**

In [7]:
# Set Frequency to Monthly
df.index.freq = 'MS'

In [8]:
# Instantiation
ses = SimpleExpSmoothing(df['Passengers'])

In [9]:
# Fit the Model
res = ses.fit(smoothing_level=0.2, optimized=False)

In [10]:
# Predict on the entire data
# Notice that the SES is one step ahead of EWMA
df['SES'] = res.predict(start=df.index[0], end=df.index[-1])
df.head()

,Passengers,EWMA,SES
Month,,,
1949-01-01,112,112.0000,112.000
1949-02-01,118,113.2000,112.000
1949-03-01,132,116.9600,113.200
1949-04-01,129,119.3680,116.960
1949-05-01,121,119.6944,119.368


In [11]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'EWMA', 'SES'])

In [12]:
# Split data into train and test
N_test = 12
train = df.iloc[:-N_test]
test = df.iloc[-N_test:]

In [13]:
# Instantiation
ses = SimpleExpSmoothing(train['Passengers'])

In [14]:
# Fit the Model
res = ses.fit()

In [15]:
# Set index for 'SES-Fitted' Column
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

In [16]:
# Calculate FittedValues and Forecast for SES-Fitted Column
df.loc[train_idx, 'SES-Fitted'] = res.fittedvalues
df.loc[test_idx, 'SES-Fitted'] = res.forecast(N_test)

In [17]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'SES-Fitted'])

In [18]:
# res parameters
res.params

{'smoothing_level': 0.995,
 'smoothing_trend': nan,
 'smoothing_seasonal': nan,
 'damping_trend': nan,
 'initial_level': 112.0,
 'initial_trend': nan,
 'initial_seasons': array([], dtype=float64),
 'use_boxcox': False,
 'lamda': None,
 'remove_bias': False}

# **Holt Model**

In [19]:
# Instantiation
holt = Holt(df['Passengers'])

In [20]:
# Fit the Model
res_h = holt.fit()

In [21]:
# Create Holt column with fittedvalues's result
df['Holt'] = res_h.fittedvalues

In [22]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt'])

In [23]:
# Instantiation
holt = Holt(train['Passengers'])

In [24]:
# Fit the Model
res_h = holt.fit()

In [25]:
# Calculate FittedValues and Forecast for Holt-Fitted Column
df.loc[train_idx, 'Holt'] = res_h.fittedvalues
df.loc[test_idx, 'Holt'] = res_h.forecast(N_test)

In [26]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt'])

# **ExponentialSmoothing (Holt-Winters) Model**

In [27]:
# Instantiation (trend='add', seasonal='add')
hw = ExponentialSmoothing(train['Passengers'], trend='add', seasonal='add', seasonal_periods=12)

In [28]:
# Fit the Model
res_hw = hw.fit()

In [29]:
# Calculate FittedValues and Forecast for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters'] = res_hw.fittedvalues
df.loc[test_idx, 'Holt-Winters'] = res_hw.forecast(N_test)

In [30]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt-Winters'])

In [31]:
# Instantiation (trend='add', seasonal='mul')
hw_2 = ExponentialSmoothing(train['Passengers'], trend='add', seasonal='mul', seasonal_periods=12)

In [32]:
# Fit the Model
res_hw_2 = hw_2.fit()

In [33]:
# Calculate FittedValues and Forecast for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters_2'] = res_hw_2.fittedvalues
df.loc[test_idx, 'Holt-Winters_2'] = res_hw_2.forecast(N_test)

In [34]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt-Winters_2'])

In [35]:
# Instantiation (trend='mul', seasonal='mul')
hw_3 = ExponentialSmoothing(train['Passengers'], trend='mul', seasonal='mul', seasonal_periods=12)

In [36]:
# Fit the Model
res_hw_3 = hw_3.fit()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning:

overflow encountered in matmul



In [37]:
# Calculate FittedValues and Forecast for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters_3'] = res_hw_3.fittedvalues
df.loc[test_idx, 'Holt-Winters_3'] = res_hw_3.forecast(N_test)

In [38]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt-Winters_3'])

In [39]:
# Plot 
px.line(df, x=df.index, y=['Passengers', 'Holt-Winters', 'Holt-Winters_2', 'Holt-Winters_3'])